Now take your Keras skills and go build another neural network. Pick your data set, but it should be one of abstract types, possibly even nonnumeric, and use Keras to make five implementations of your network. Compare them both in computational complexity as well as in accuracy and given that tradeoff decide which one you like best.

Your dataset should be sufficiently large for a neural network to perform well (samples should really be in the thousands here) and try to pick something that takes advantage of neural networks’ ability to have both feature extraction and supervised capabilities, so don’t pick something with an easy to consume list of features already generated for you (though neural networks can still be useful in those contexts).

Note that if you want to use an unprocessed image dataset, scikit-image is a useful package for converting to importable numerics.

In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
# Import the dataset
from keras.datasets import mnist

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

In [3]:
from keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [10]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(404, 13) (404, 51) (102, 13) (102,)


In [21]:
print(y_test)

[  7.2  18.8  19.   27.   22.2  24.5  31.2  22.9  20.5  23.2  18.6  14.5
  17.8  50.   20.8  24.3  24.2  19.8  19.1  22.7  12.   10.2  20.   18.5
  20.9  23.   27.5  30.1   9.5  22.   21.2  14.1  33.1  23.4  20.1   7.4
  15.4  23.8  20.1  24.5  33.   28.4  14.1  46.7  32.5  29.6  28.4  19.8
  20.2  25.   35.4  20.3   9.7  14.5  34.9  26.6   7.2  50.   32.4  21.6
  29.8  13.1  27.5  21.2  23.1  21.9  13.   23.2   8.1   5.6  21.7  29.6
  19.6   7.   26.4  18.9  20.9  28.1  35.4  10.2  24.3  43.1  17.6  15.4
  16.2  27.1  21.4  21.5  22.4  25.   16.6  18.6  22.   42.8  35.1  21.5
  36.   21.9  24.1  50.   26.7  25. ]


### Multi-Layer Perceptron Model

In [19]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.

model.add(Dense(64, input_shape=(13,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64,))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(51,))

model.summary()

# Compile the model to put it all together.
model.compile(loss='mean_squared_error',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 64)                896       
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 51)                3315      
Total params: 8,371
Trainable params: 8,371
Non-trainable params: 0
_________________________________________________________________


In [23]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1)
                    #validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
404/404 [==============================] - 0s - loss: 6313.4673 - acc: 0.0124      
Epoch 2/10
404/404 [==============================] - 0s - loss: 2500.7015 - acc: 0.0173     
Epoch 3/10
404/404 [==============================] - 0s - loss: 1902.9917 - acc: 0.0173     
Epoch 4/10
404/404 [==============================] - 0s - loss: 1597.6945 - acc: 0.0149     
Epoch 5/10
404/404 [==============================] - 0s - loss: 1393.3483 - acc: 0.0173     
Epoch 6/10
404/404 [==============================] - 0s - loss: 1310.6776 - acc: 0.0149     
Epoch 7/10
404/404 [==============================] - 0s - loss: 1160.8681 - acc: 0.0198         
Epoch 8/10
404/404 [==============================] - 0s - loss: 1085.2638 - acc: 0.0173     
Epoch 9/10
404/404 [==============================] - 0s - loss: 942.0034 - acc: 0.0347     
Epoch 10/10
404/404 [==============================] - 0s - loss: 839.9720 - acc: 0.0149     


ValueError: Error when checking target: expected dense_18 to have shape (None, 51) but got array with shape (102, 1)

### Convolutional Neural Network

In [ ]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Hierarchical Recurrent Neural Networks

In [ ]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])